---
title: "Introduction to Google Earth Engine Authentication and Common Libraries"
author: "Your Name"
date: "October 26, 2023"
---

## Introduction

This notebook provides a basic introduction to authenticating with Google Earth Engine (GEE) and installing/importing common libraries used in GEE workflows, including `geemap`.  `geemap` simplifies many GEE tasks and provides interactive mapping capabilities.

## 1. Authentication with Google Earth Engine

Before you can use GEE, you need to authenticate your account. There are several ways to do this:

*   **Browser Authentication (Recommended):** This is the easiest method.  GEE will open a browser window to allow you to log in with your Google account.
*   **Service Account:**  More suitable for automated scripts and server-side applications. Requires creating a service account in the Google Cloud Console and downloading credentials.

### 1.1 Browser Authentication (using `ee.Initialize()`)

The following code snippet initializes the Earth Engine API and authenticates your account using browser authentication.  Run this cell *first* to authenticate.

In [ ]:
import ee
import geemap as geemap

# Initialize Earth Engine (authenticate if needed)
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()
# Print a confirmation message to let us know the initialization was successful
print('Earth Engine initialized.')

**Important:**  You may be prompted to authorize access to your Google account. Follow the instructions in the browser window.  Once authorized, you can proceed with other GEE operations.

## 2. Installing and Importing Common Libraries

Before we install specific libraries for working with Google Earth Engine, we need to ensure we have the core Earth Engine API installed.

**Note:** In Jupyter notebooks, the `!` symbol at the start of a line tells Python to run the command in your computer's terminal/command line rather than as Python code. This is how we install packages directly from within a notebook.

### 2.0 Installing the Earth Engine API

The `earthengine-api` package provides the core functionality to connect to and use Google Earth Engine from Python.

**Installation:** Run the following command in a code cell:

```bash
!pip install earthengine-api
```

The `pip install` command downloads and installs the package, and the `!` at the beginning runs this as a system command in your terminal.

**Importing:** After installation, you can import and use Earth Engine with:

```python
import ee
```

(We already did this in Cell 1 above.)

In [ ]:
# !pip install earthengine-api # If not already installed, uncomment and run this line to install the Earth Engine Python API

In [ ]:
import ee  # Import the Earth Engine Python library (ee), which allows us to use Google Earth Engine features

# Initialize Earth Engine (authenticate via browser).
# This step connects your Python environment to Google Earth Engine's servers
# If this is your first time, a browser window will open asking you to log in with your Google account
# After logging in once, your credentials are saved for future sessions

ee.Initialize()

# Print a confirmation message to let us know the initialization was successful
print('Earth Engine initialized.')

### 2.1 `geemap`

`geemap` is a Python package that simplifies many GEE tasks, including:

*   Interactive mapping
*   Data visualization
*   Image processing
*   Exporting data

In [ ]:
# !pip install geemap # Uncomment and install the geemap package for enhanced mapping and visualization capabilities with Earth Engine

In [ ]:
import geemap # Import the geemap library for mapping and visualization
Map = geemap.Map()  # Create a map object for visualization
Map # Display the map in the notebook

### 2.2 Other Useful Libraries

*   **`matplotlib`:** For creating static plots and visualizations.
*   **`numpy`:**  For numerical operations and array manipulation.
*   **`pandas`:** For data analysis and handling tabular data.

```python
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
```


In [ ]:
import matplotlib.pyplot as plt # Import Matplotlib for plotting and visualization
import numpy as np # Import NumPy for numerical operations
import pandas as pd# Import Pandas for data manipulation and analysis

## 3. Basic GEE Operations with `geemap`

### 3.1 Loading an Image

Let's load a Landsat 8 image and display it on the map.

In [ ]:
# Load a Landsat 9 surface reflectance image.
image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_039033_20220108')

# Display the image on the map.
Map.centerObject(image, zoom=10)

Map.addLayer(image, {'min': 0, 'max': 40000, 'bands': ['SR_B5', 'SR_B4', 'SR_B1']}, 'Landsat 9')
Map.addLayer(image.select('SR_B4'), {'min':0, 'max': 30000}, 'Red Band')
Map.addLayer(image.select('SR_B5'), {'min':0, 'max': 30000}, 'NIR Band')
Map.addLayer(image.select('SR_B7'), {'min':0, 'max': 30000}, 'SWIR Band')
Map.addLayer(image.select('SR_B6'), {'min':0, 'max': 30000}, 'Thermal Band')
Map.addLayer(image.select('SR_B2'), {'min':0, 'max': 30000}, 'Blue Band')
Map.addLayer(image.select('SR_B1'), {'min':0, 'max': 30000}, 'Green Band')
Map.addLayer(image.select('SR_B3'), {'min':0, 'max': 30000}, 'Red Band')
Map.addLayer(image.select('ST_B10'), {'min':0, 'max': 30000}, 'SWIR Band')

### 3.1 Loading an Image

Let's load a Landsat 9 surface reflectance image and display it on the map.

In [ ]:
# Load a Landsat 9 surface reflectance image collection.
collection = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# Filter the collection to get a single image (e.g., the first one).
image = collection.first()

# Display the image on the map.
Map.centerObject(image, zoom=10)

# Add layers for different bands with appropriate min/max values.
Map.addLayer(image.select('SR_B4'), {'min': 0, 'max': 30000}, 'Red Band')
Map.addLayer(image.select('SR_B5'), {'min': 0, 'max': 30000}, 'NIR Band')
Map.addLayer(image.select('SR_B6'), {'min': 0, 'max': 30000}, 'SWIR Band')
Map.addLayer(image.select('SR_B7'), {'min': 0, 'max': 30000}, 'SWIR Band')
Map.addLayer(image.select('SR_B2'), {'min': 0, 'max': 30000}, 'Blue Band')
Map.addLayer(image.select('SR_B3'), {'min': 0, 'max': 30000}, 'Green Band')
Map.addLayer(image.select('SR_B1'), {'min': 0, 'max': 30000}, 'Ultra Blue Band')
Map.addLayer(image.select('ST_B10'), {'min': 273, 'max': 313}, 'Thermal Band (K)')

### 3.2 Visualizing with False Color Composites

We can create false color composites to highlight different features in the image. For example, a common composite is to use NIR (Band 5), Red (Band 4), and Green (Band 3).

In [ ]:
# Create a false color composite.
composite = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").first().normalizedDifference(
    ['SR_B5', 'SR_B4']).clip(Map.getCenterObject().geometry())

# Add the composite to the map.
Map.addLayer(composite, {'min': -0.2, 'max': 0.3}, 'NDVI')

### 3.3 Calculating NDVI (Normalized Difference Vegetation Index)

NDVI is a simple index to measure vegetation greenness. It's calculated as `(NIR - Red) / (NIR + Red)`.

In [ ]:
# Calculate NDVI.
ndvi = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").first().normalizedDifference(
    ['SR_B5', 'SR_B4']).clip(Map.getCenterObject().geometry())

# Add NDVI to the map.
Map.addLayer(ndvi, {'min': -0.2, 'max': 0.3}, 'NDVI')



In [ ]:
Map